In [ ]:
!pip install transformers sentencepiece torch pandas tqdm konlpy scikit-learn -q
!git clone https://github.com/park1200656/KnuSentiLex.git knusentilexdownload 2>/dev/null || echo "KnuSentiLex already present"

In [ ]:
# import pandas as pd
# import numpy as np
# from tqdm import tqdm

# # ------------------------------------------------------------
# # 1️⃣  댓글 데이터 (예: youtube_comments)
# # ------------------------------------------------------------
# # 반드시 'text' 컬럼이 있어야 합니다.
# # 예시)
# # youtube_comments = pd.read_csv("youtube_comments.csv")
# # columns = ['video_id','text','likeCount','published_at']

# # ------------------------------------------------------------
# # 2️⃣  감정 분석 함수 (기존 코드에서 가져옴)
# # ------------------------------------------------------------

# def fused_sent_score(text: str) -> dict:
#     """KR-FinBERT + Kiwi + KnuSentiLex 간단 결합 버전"""
#     lex = lexicon_sent_score(text)
#     fin = finbert_sent_score(text)
#     fused = 0.7 * fin["finbert_score"] + 0.3 * lex["lex_score"]

#     if fused > 0.2:
#         label = "긍정"
#     elif fused < -0.2:
#         label = "부정"
#     else:
#         label = "중립"

#     return {"text_label": label, "fused_score": float(np.clip(fused, -1, 1))}

# # ------------------------------------------------------------
# # 3️⃣  댓글별 감정 라벨링 수행
# # ------------------------------------------------------------

# def add_text_label(df, text_col="text"):
#     results = []
#     for text in tqdm(df[text_col], desc="댓글 감정 분석중"):
#         try:
#             result = fused_sent_score(str(text))
#         except Exception as e:
#             result = {"text_label": "오류", "fused_score": 0.0}
#         results.append(result)

#     res_df = pd.DataFrame(results)
#     df = pd.concat([df.reset_index(drop=True), res_df], axis=1)
#     return df

# # ------------------------------------------------------------
# # 4️⃣  실행
# # ------------------------------------------------------------
# youtube_comments_labeled = add_text_label(youtube_comments)

# # 결과 확인
# print(youtube_comments_labeled.head())


댓글 감정 분석중: 100%|██████████| 6/6 [00:00<00:00, 23.28it/s]

  video_id                text text_label  fused_score
0       v1   역시 대단하다 또 유출이네 ㅋㅋ         부정    -0.637439
1       v1  서비스 좋아요. 개선도 빨라졌고요         중립     0.078816
2       v1         아 이건 좀 아닌 듯         중립    -0.001716
3       v2        업데이트 나쁘지 않네요         중립    -0.000021
4       v2          최악이다 진심 실망         부정    -0.697706


In [ ]:
# # ============================================================
# # 1️⃣ 라이브러리 설치
# # ============================================================
# # (colab 또는 로컬 환경에서 1회만 실행)
# # !pip install transformers torch kiwipiepy --upgrade

# # ============================================================
# # 2️⃣ 라이브러리 임포트
# # ============================================================
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# import torch
# from kiwipiepy import Kiwi

# # ============================================================
# # 3️⃣ 모델 및 토크나이저 불러오기
# # ============================================================
# MODEL_NAME = "alsgyu/sentiment-analysis-fine-tuned-model"

# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

# # ============================================================
# # 4️⃣ 감정 예측 함수 정의
# # ============================================================
# def predict_sentiment(text):
#     inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
#     outputs = model(**inputs)
#     probs = torch.nn.functional.softmax(outputs.logits, dim=-1)[0]
#     label_id = torch.argmax(probs).item()

#     # 모델 레이블 정의 (모델 카드에 따라 달라질 수 있음)
#     labels = ["부정", "중립", "긍정"] if probs.shape[0] == 3 else ["부정", "긍정"]

#     return {
#         "text": text,
#         "label": labels[label_id],
#         "confidence": round(float(probs[label_id]), 3),
#         "scores": {labels[i]: round(float(p), 3) for i, p in enumerate(probs)}
#     }

# # ============================================================
# # 5️⃣ 테스트 문장
# # ============================================================
# samples = [
#     "와, 진짜 천재시네요. 이렇게 완벽한 실수는 처음 봅니다.",
#     "그렇게 잘 될 줄 알았습니다. 다음번엔 노벨상이라도 타시겠어요.",
#     "역시 기대를 저버리지 않으시네요.",
#     "작성자에 의해 삭제된 댓글입니다.",
#     "당하기만 하는 나라"
# ]

# # ============================================================
# # 6️⃣ 실행
# # ============================================================
# results = [predict_sentiment(text) for text in samples]

# for r in results:
#     print(f"\n문장: {r['text']}")
#     print(f"→ 감정: {r['label']} ({r['confidence']})")
#     print(f"세부 점수: {r['scores']}")


tokenizer_config.json: 0.00B [00:00, ?B/s]

c:\Users\speec\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\speec\.cache\huggingface\hub\models--alsgyu--sentiment-analysis-fine-tuned-model. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]


문장: 와, 진짜 천재시네요. 이렇게 완벽한 실수는 처음 봅니다.
→ 감정: 긍정 (0.915)
세부 점수: {'부정': 0.01, '중립': 0.076, '긍정': 0.915}

문장: 그렇게 잘 될 줄 알았습니다. 다음번엔 노벨상이라도 타시겠어요.
→ 감정: 긍정 (0.953)
세부 점수: {'부정': 0.004, '중립': 0.043, '긍정': 0.953}

문장: 역시 기대를 저버리지 않으시네요.
→ 감정: 중립 (0.481)
세부 점수: {'부정': 0.149, '중립': 0.481, '긍정': 0.37}

문장: 작성자에 의해 삭제된 댓글입니다.
→ 감정: 긍정 (0.514)
세부 점수: {'부정': 0.194, '중립': 0.292, '긍정': 0.514}

문장: 당하기만 하는 나라
→ 감정: 부정 (0.594)
세부 점수: {'부정': 0.594, '중립': 0.294, '긍정': 0.112}


C:\Users\speec\AppData\Local\Temp\ipykernel_25632\2955265296.py:37: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\autograd\generated\python_variable_methods.cpp:836.)
  "confidence": round(float(probs[label_id]), 3),


In [7]:
!git clone https://github.com/park1200656/KnuSentiLex.git knusentilexdownload

Cloning into 'knusentilexdownload'...


In [ ]:
2모델 비교 실험 
HYBRID_MODEL_CANDIDATES = [
    "snunlp/KR-FinBERT",
    "snunlp/KR-FinBert-Sentiment",
    "beomi/KcELECTRA-base"
]
ALS_MODEL_NAME = "alsgyu/sentiment-analysis-fine-tuned-model"


In [ ]:
# # ============================================================
# # KnuSentiLex 사전 + 하이브리드 감정분석 통합 코드
# # ============================================================

# import os
# import pandas as pd
# import torch
# from kiwipiepy import Kiwi
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
# from typing import Dict, List, Tuple

# # -----------------------------
# # 설정
# # -----------------------------
# KNU_REPO_URL = "https://github.com/park1200656/KnuSentiLex.git"
# # ── 위 URL은 예시이며 실제 경로는 저장소 구조 확인 필요
# HYBRID_MODEL_CANDIDATES = [
#     "snunlp/KR-FinBERT",
#     "snunlp/KR-FinBert-Sentiment",
#     "beomi/KcELECTRA-base"
# ]
# ALS_MODEL_NAME = "alsgyu/sentiment-analysis-fine-tuned-model"

# # -----------------------------
# # 1) 사전 다운로드 및 로드
# # -----------------------------
# def download_knu(path: str) -> None:
#     os.system(f"wget -O {path} {KNU_REPO_URL}")

# def load_knu_lexicon(path: str) -> Dict[str, float]:
#     lex = {}
#     # 파일이 탭 또는 공백 구분일 가능성
#     df = pd.read_csv(path, sep="\t", header=None, names=["word", "polarity"], encoding="utf-8")
#     for _, r in df.iterrows():
#         word = str(r["word"]).strip()
#         try:
#             score = float(r["polarity"])
#         except:
#             continue
#         lex[word] = score
#     return lex

# # 파일 경로 찾기 - 여러 경로 시도
# possible_paths = [
#     "knusentilexdownload/SentiWord_Dict.txt",
#     "knusentilexdownload/KnuSentiLex/SentiWord_Dict.txt",
#     "KnuSentiLex_SentiWord_Dict.txt"
# ]

# LEXICON_PATH = None
# for path in possible_paths:
#     if os.path.exists(path):
#         LEXICON_PATH = path
#         print(f"✓ 파일 찾음: {path}")
#         break

# if LEXICON_PATH is None:
#     print("⚠ SentiWord_Dict.txt를 찾을 수 없습니다!")
#     print(f"가능한 경로들:")
#     for p in possible_paths:
#         print(f"  - {p}")
#     # 기본 경로로 설정
#     LEXICON_PATH = possible_paths[0]

# LEXICON = load_knu_lexicon(LEXICON_PATH)
# print(f"[LOAD] KnuSentiLex entries: {len(LEXICON)}")

# # -----------------------------
# # 2) 형태소 분석기 초기화
# # -----------------------------
# kiwi = Kiwi()

# # -----------------------------
# # 3) 모델 로딩 함수
# # -----------------------------
# def load_classifier(model_name: str) -> Tuple[AutoTokenizer, AutoModelForSequenceClassification, List[str]]:
#     tok = AutoTokenizer.from_pretrained(model_name)
#     mdl = AutoModelForSequenceClassification.from_pretrained(model_name)
#     with torch.no_grad():
#         dummy = tok("테스트 문장", return_tensors="pt", truncation=True, padding=True)
#         out = mdl(**dummy)
#     n = out.logits.shape[-1]
#     if n == 2:
#         labels = ["부정", "긍정"]
#     elif n == 3:
#         labels = ["부정", "중립", "긍정"]
#     else:
#         labels = [f"class_{i}" for i in range(n)]
#     return tok, mdl, labels

# def try_load_any(candidates: List[str]) -> Tuple[str, AutoTokenizer, AutoModelForSequenceClassification, List[str]]:
#     for name in candidates:
#         try:
#             tok, mdl, lbls = load_classifier(name)
#             print(f"[LOAD] base model loaded: {name} -> {lbls}")
#             return name, tok, mdl, lbls
#         except Exception as e:
#             print(f"[WARN] failed to load {name}: {e}")
#             continue
#     raise RuntimeError("No base model could be loaded.")

# BASE_NAME, base_tok, base_mdl, base_labels = try_load_any(HYBRID_MODEL_CANDIDATES)
# als_tok, als_mdl, als_labels = load_classifier(ALS_MODEL_NAME)
# print(f"[LOAD] als model loaded: {ALS_MODEL_NAME} -> {als_labels}")

# # -----------------------------
# # 4) Lexicon 점수 함수
# # -----------------------------
# def lexicon_score(text: str) -> float:
#     tokens = [t.form for t in kiwi.tokenize(text)]
#     return sum(LEXICON.get(tok, 0.0) for tok in tokens)

# def sarcasm_flag(text: str, lscore: float) -> bool:
#     tokens = [t.form for t in kiwi.tokenize(text)]
#     positive_markers = {"천재", "대단", "역시", "잘", "노벨상"}
#     if any(tok in positive_markers for tok in tokens) and lscore < -0.3:
#         return True
#     if text.count("...") >= 2 or ("?" in text and "!" in text):
#         return True
#     return False

# # -----------------------------
# # 5) 예측 함수
# # -----------------------------
# @torch.no_grad()
# def predict_hf(tok, mdl, labels, text: str) -> Tuple[str, float, Dict[str,float]]:
#     inputs = tok(text, return_tensors="pt", truncation=True, padding=True)
#     out = mdl(**inputs)
#     probs = torch.softmax(out.logits, dim=-1)[0].cpu().numpy().tolist()
#     idx = int(torch.argmax(out.logits, dim=-1).item())
#     return labels[idx], float(probs[idx]), {labels[i]: round(probs[i],3) for i in range(len(labels))}

# def hybrid_predict(text: str, w_model: float = 0.7, w_lex: float = 0.3) -> Dict:
#     lscore = lexicon_score(text)
#     base_label, base_conf, base_dist = predict_hf(base_tok, base_mdl, base_labels, text)
#     signed = (base_dist.get("긍정",0) - base_dist.get("부정",0))
#     final_score = w_model * signed + w_lex * lscore
#     if "삭제된 댓글" in text or "부적절한 표현" in text:
#         label = "기타"
#     elif sarcasm_flag(text, lscore):
#         label = "반어/비꼼"
#     else:
#         if final_score > 0.2:
#             label = "긍정"
#         elif final_score < -0.2:
#             label = "분노(강부정)" if lscore < -1.0 else "부정"
#         else:
#             label = "중립"
#     return {
#         "label": label,
#         "score": round(final_score,3),
#         "lex_score": round(lscore,3),
#         "base_label": base_label,
#         "base_conf": round(base_conf,3),
#         "base_dist": base_dist
#     }

# # -----------------------------
# # 6) 비교 실행
# # -----------------------------
# def compare_models(texts: List[str]) -> None:
#     for t in texts:
#         hy = hybrid_predict(t)
#         als_label, als_conf, als_dist = predict_hf(als_tok, als_mdl, als_labels, t)
#         print("\n문장:", t)
#         print(">> HYBRID :", hy["label"], f"(score={hy['score']}, lex={hy['lex_score']})", f"base={hy['base_label']}")
#         print(">> ALS     :", als_label, f"(conf={als_conf})", "dist=", als_dist)

# if __name__ == "__main__":
#     sample_comments = [
#         "여휴 이러다가 조만간 모든국민 통신 마비되는건가 의심스럽다 어디로 가야하나",
#         "저거이 10년전에예정됐던겁니다.카카오만문제냐는식사론글로 저는 현장에서 대각제되었던거고 인증까지",
#         "국내 거주중인 홍*인 친구 부정하고 입국을 막아라. 더넘어 미국인도 추방하고.. 정부 뭐하나? 실망.",
#         "SK랑 LG도 보안 강화해라.",
#         "이정현이 일 사회제재 하라고 부추기는거 같다",
#         "진짜 우리나라 통신요금은 받아먹을거 다 받아먹고 기술개발이나 유지보수는 뭐같이 안하는듯",
#         "민주 정부 시절에 해서 대박이 났네 ㅋㅋㅋㅋ",
#         "KT가 더 악질이다 SK 하수인 보는듯 해서 신경써야겠다는 생각을 진짜 누군말까도 안한거가?",
#         "시 당연히 정부에서도 아주 개판이군.",
#         "돈만 받아 쳐먹을욕지 보상해줄 생각도없는 개그지 같은 놈들"
#     ]
#     compare_models(sample_comments)


✓ 파일 찾음: knusentilexdownload/SentiWord_Dict.txt
[LOAD] KnuSentiLex entries: 14852


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at snunlp/KR-FinBERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[LOAD] base model loaded: snunlp/KR-FinBERT -> ['부정', '긍정']
[LOAD] als model loaded: alsgyu/sentiment-analysis-fine-tuned-model -> ['부정', '중립', '긍정']

문장: 여휴 이러다가 조만간 모든국민 통신 마비되는건가 의심스럽다 어디로 가야하나
>> HYBRID : 분노(강부정) (score=-0.428, lex=-2.0) base=긍정
>> ALS     : 부정 (conf=0.6244485974311829) dist= {'부정': 0.624, '중립': 0.267, '긍정': 0.108}

문장: 저거이 10년전에예정됐던겁니다.카카오만문제냐는식사론글로 저는 현장에서 대각제되었던거고 인증까지
>> HYBRID : 중립 (score=0.13, lex=0.0) base=긍정
>> ALS     : 긍정 (conf=0.5859249830245972) dist= {'부정': 0.146, '중립': 0.268, '긍정': 0.586}

문장: 국내 거주중인 홍*인 친구 부정하고 입국을 막아라. 더넘어 미국인도 추방하고.. 정부 뭐하나? 실망.
>> HYBRID : 분노(강부정) (score=-0.47, lex=-2.0) base=긍정
>> ALS     : 부정 (conf=0.6572269797325134) dist= {'부정': 0.657, '중립': 0.227, '긍정': 0.116}

문장: SK랑 LG도 보안 강화해라.
>> HYBRID : 중립 (score=0.102, lex=0.0) base=긍정
>> ALS     : 긍정 (conf=0.4922817349433899) dist= {'부정': 0.207, '중립': 0.3, '긍정': 0.492}

문장: 이정현이 일 사회제재 하라고 부추기는거 같다
>> HYBRID : 중립 (score=0.109, lex=0.0) base=긍정
>> ALS     : 부정 (conf=0.457269251346588

In [9]:
# ========== 파일 경로 확인 및 수정 ==========
import os
import glob

print("[파일 경로 확인]")
base_path = "knusentilexdownload"

# 다운로드된 파일 확인
all_files = glob.glob(f"{base_path}/**/*.txt", recursive=True)
print(f"\n찾은 .txt 파일: {len(all_files)}개")
for f in all_files[:10]:
    print(f"  - {f}")

# 주요 파일 확인
key_files = [
    os.path.join(base_path, "pos_pol_word.txt"),
    os.path.join(base_path, "neg_pol_word.txt"),
    os.path.join(base_path, "KnuSentiLex", "pos_pol_word.txt"),
    os.path.join(base_path, "KnuSentiLex", "neg_pol_word.txt"),
]

print("\n[주요 파일 존재 여부]")
for f in key_files:
    exists = "✓" if os.path.exists(f) else "✗"
    print(f"{exists} {f}")


[파일 경로 확인]

찾은 .txt 파일: 10개
  - knusentilexdownload\neg_pol_word.txt
  - knusentilexdownload\obj_unknown_pol_word.txt
  - knusentilexdownload\pos_pol_word.txt
  - knusentilexdownload\ReadMe.txt
  - knusentilexdownload\SentiWord_Dict.txt
  - knusentilexdownload\KnuSentiLex\neg_pol_word.txt
  - knusentilexdownload\KnuSentiLex\obj_unknown_pol_word.txt
  - knusentilexdownload\KnuSentiLex\pos_pol_word.txt
  - knusentilexdownload\KnuSentiLex\ReadMe.txt
  - knusentilexdownload\KnuSentiLex\SentiWord_Dict.txt

[주요 파일 존재 여부]
✓ knusentilexdownload\pos_pol_word.txt
✓ knusentilexdownload\neg_pol_word.txt
✓ knusentilexdownload\KnuSentiLex\pos_pol_word.txt
✓ knusentilexdownload\KnuSentiLex\neg_pol_word.txt


In [12]:
# ============================================================
# KnuSentiLex 사전 + 하이브리드 감정분석 (교정/강화 버전)
# ============================================================

import os
import subprocess
import pandas as pd
import torch
from typing import Dict, List, Tuple
from kiwipiepy import Kiwi
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# -----------------------------
# 0) 설정
# -----------------------------
# A안: raw 파일 직접 받기 (repo knu_senti_dict에 raw 파일이 존재)
RAW_TXT_URLS = [
    "https://raw.githubusercontent.com/park1200656/knu_senti_dict/master/SentiWord_Dict.txt",
    "https://raw.githubusercontent.com/park1200656/KnuSentiLex/master/SentiWord_Dict.txt",  # 대체 경로 (없을 수도 있음)
]
LEXICON_LOCAL = "SentiWord_Dict.txt"

# B안: git clone 경로
GIT_REPO = "https://github.com/park1200656/KnuSentiLex.git"
GIT_DIR  = "KnuSentiLex"

# 분류 모델 후보
BASE_CANDIDATES = [
    "snunlp/KR-FinBERT",                    # 최우선 (없으면 다음)
    "nlpai-lab/kcelectra-base-kor-finetuned-nsmc",  # 대체 1
    "beomi/KcELECTRA-base"                  # 대체 2
]
ALS_MODEL = "alsgyu/sentiment-analysis-fine-tuned-model"

# -----------------------------
# 1) KNU 사전 확보: RAW → GIT 순서로 시도
# -----------------------------
def ensure_lexicon_file() -> str:
    # 1) raw 다운로드 시도
    for url in RAW_TXT_URLS:
      try:
        rc = subprocess.call(["curl", "-L", "-o", LEXICON_LOCAL, url], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        if rc == 0 and os.path.exists(LEXICON_LOCAL) and os.path.getsize(LEXICON_LOCAL) > 0:
            print(f"[KNU] downloaded raw file from: {url}")
            return LEXICON_LOCAL
      except Exception:
        pass

    # 2) git clone 후 파일 찾기
    if not os.path.exists(GIT_DIR):
        subprocess.call(["git", "clone", "--depth", "1", GIT_REPO], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    # 후보 경로들
    candidates = [
        os.path.join(GIT_DIR, "SentiWord_Dict.txt"),
        os.path.join(GIT_DIR, "data", "SentiWord_Dict.txt"),
        os.path.join(GIT_DIR, "KnuSentiLex-master", "SentiWord_Dict.txt"),
    ]
    for p in candidates:
        if os.path.exists(p) and os.path.getsize(p) > 0:
            print(f"[KNU] found file: {p}")
            return p

    # 실패 시 종료
    raise FileNotFoundError("KNU SentiWord_Dict.txt not found. raw URL/Git clone 모두 실패")

def load_knu_lexicon(path: str) -> Dict[str, float]:
    # TSV, 2열(word/emoticon, polarity). BOM·잡문자 대응
    df = pd.read_csv(
        path,
        sep="\t",
        header=None,
        names=["word", "polarity"],
        encoding="utf-8",
        engine="python",
        on_bad_lines="skip"
    )
    # 숫자 변환, NaN drop
    df["polarity"] = pd.to_numeric(df["polarity"], errors="coerce")
    df = df.dropna(subset=["word", "polarity"])
    # 공백 정리
    df["word"] = df["word"].astype(str).str.strip()
    # 딕셔너리화
    return dict(zip(df["word"], df["polarity"]))

LEXICON_PATH = ensure_lexicon_file()
LEXICON: Dict[str, float] = load_knu_lexicon(LEXICON_PATH)
print(f"[KNU] entries: {len(LEXICON):,}")

# -----------------------------
# 2) 형태소 분석기
# -----------------------------
kiwi = Kiwi()

def normalize_token(tok: str) -> str:
    # 필요 시 추가 규칙(소문자화, 특수문자 예외 처리 등)
    return tok.strip()

def lexicon_score(text: str) -> float:
    # 이모티콘/표정 등 KNU에 그대로 있는 토큰은 공백 분할로도 잡힘
    tokens = [t.form for t in kiwi.tokenize(text)]
    score = 0.0
    for t in tokens:
        key = normalize_token(t)
        if key in LEXICON:
            score += float(LEXICON[key])
    # 원문 그대로 매칭(이모티콘 등)을 위해 한 번 더 가벼운 스캔
    for raw_piece in text.split():
        key = normalize_token(raw_piece)
        score += float(LEXICON.get(key, 0.0))
    return score

# -----------------------------
# 3) 분류기 로더
# -----------------------------
def load_classifier(name: str):
    tok = AutoTokenizer.from_pretrained(name)
    mdl = AutoModelForSequenceClassification.from_pretrained(name)
    with torch.no_grad():
        dummy = tok("테스트", return_tensors="pt", truncation=True, padding=True)
        n_cls = mdl(**dummy).logits.shape[-1]
    if n_cls == 2:
        labels = ["부정", "긍정"]
    elif n_cls == 3:
        labels = ["부정", "중립", "긍정"]
    else:
        labels = [f"class_{i}" for i in range(n_cls)]
    return tok, mdl, labels

def try_load_any(cands: List[str]):
    last = None
    for n in cands:
        try:
            tok, mdl, labels = load_classifier(n)
            print(f"[LOAD] base: {n} -> {labels}")
            return n, tok, mdl, labels
        except Exception as e:
            last = e
            continue
    raise RuntimeError(f"no base model found; last error: {last}")

BASE_NAME, base_tok, base_mdl, base_labels = try_load_any(BASE_CANDIDATES)
als_tok, als_mdl, als_labels = load_classifier(ALS_MODEL)
print(f"[LOAD] als  : {ALS_MODEL} -> {als_labels}")

# -----------------------------
# 4) 예측 + 하이브리드
# -----------------------------
@torch.no_grad()
def predict_logits(tok, mdl, labels, text: str):
    x = tok(text, return_tensors="pt", truncation=True, padding=True)
    logits = mdl(**x).logits
    probs = torch.softmax(logits, dim=-1)[0].cpu().numpy().tolist()
    idx = int(torch.argmax(logits, dim=-1).item())
    return labels[idx], float(probs[idx]), {labels[i]: float(p) for i, p in enumerate(probs)}

def sarcasm_flag(text: str, lscore: float) -> bool:
    # 간단 규칙: 칭찬표현+부정 lexicon, 과도한 구두점, 반문
    pos_markers = {"천재","대단","역시","완벽","잘했다","축하"}
    has_pos = any(w in text for w in pos_markers)
    punct = (text.count("...") >= 2) or ("?!" in text)
    return (has_pos and lscore < -0.3) or punct

def signed_from_dist(dist: Dict[str, float]) -> float:
    p_pos = dist.get("긍정", 0.0)
    p_neg = dist.get("부정", 0.0)
    p_neu = dist.get("중립", 0.0)
    signed = p_pos - p_neg
    # 중립이 매우 크면 신호를 약화
    if p_neu > max(p_pos, p_neg):
        signed *= (1.0 - p_neu)
    return signed

def hybrid_predict(text: str, w_model=0.7, w_lex=0.3):
    lscore = lexicon_score(text)
    base_label, base_conf, base_dist = predict_logits(base_tok, base_mdl, base_labels, text)
    signed = signed_from_dist(base_dist)
    final = w_model * signed + w_lex * lscore

    if "삭제된 댓글" in text or "부적절한 표현" in text:
        label = "기타"
    elif sarcasm_flag(text, lscore):
        label = "반어/비꼼"
    else:
        if final > 0.2:
            label = "긍정"
        elif final < -0.2:
            label = "분노(강부정)" if (lscore < -1.0 or base_dist.get("부정",0) > 0.85) else "부정"
        else:
            label = "중립"

    return {
        "hy_label": label,
        "hy_score": round(final, 3),
        "lex": round(lscore, 3),
        "base_label": base_label,
        "base_conf": round(base_conf, 3),
        "base_dist": {k: round(v, 3) for k,v in base_dist.items()}
    }

def compare_models(texts: List[str]):
    rows = []
    for t in texts:
        hy = hybrid_predict(t)
        als_label, als_conf, als_dist = predict_logits(als_tok, als_mdl, als_labels, t)
        rows.append({
            "text": t,
            "HY_label": hy["hy_label"],
            "HY_score": hy["hy_score"],
            "HY_lex": hy["lex"],
            "HY_base": f"{hy['base_label']}({hy['base_conf']})",
            "ALS_label": als_label,
            "ALS_conf": round(als_conf,3),
            "ALS_dist": {k: round(v,3) for k,v in als_dist.items()}
        })
    df = pd.DataFrame(rows, columns=["text","HY_label","HY_score","HY_lex","HY_base","ALS_label","ALS_conf","ALS_dist"])
    pd.set_option("display.max_colwidth", 180)
    print(df.to_string(index=False))

if __name__ == "__main__":
    sample = [
        "와, 진짜 천재시네요. 이렇게 완벽한 실수는 처음 봅니다.",
        "교체합시다. 너무 찝찝하잖아요",
        "작성자에 의해 삭제된 댓글입니다.",
        "두번다시 케이티 안쓴다",
        "보이스피싱하고 사기꾼광고 이런문제들을 엄격하게 다룰수 있으면 좋겠다~"
    ]
    compare_models(sample)


[KNU] downloaded raw file from: https://raw.githubusercontent.com/park1200656/knu_senti_dict/master/SentiWord_Dict.txt
[KNU] entries: 14,841


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at snunlp/KR-FinBERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[LOAD] base: snunlp/KR-FinBERT -> ['부정', '긍정']
[LOAD] als  : alsgyu/sentiment-analysis-fine-tuned-model -> ['부정', '중립', '긍정']
                                  text HY_label  HY_score  HY_lex   HY_base ALS_label  ALS_conf                                ALS_dist
      와, 진짜 천재시네요. 이렇게 완벽한 실수는 처음 봅니다.       중립    -0.092     0.0 부정(0.566)        긍정     0.915  {'부정': 0.01, '중립': 0.076, '긍정': 0.915}
                      교체합시다. 너무 찝찝하잖아요       중립    -0.061     0.0 부정(0.544)        부정     0.775 {'부정': 0.775, '중립': 0.198, '긍정': 0.027}
                    작성자에 의해 삭제된 댓글입니다.       기타    -0.080     0.0 부정(0.557)        긍정     0.514 {'부정': 0.194, '중립': 0.292, '긍정': 0.514}
                          두번다시 케이티 안쓴다       중립    -0.054     0.0 부정(0.539)        긍정     0.393 {'부정': 0.382, '중립': 0.225, '긍정': 0.393}
보이스피싱하고 사기꾼광고 이런문제들을 엄격하게 다룰수 있으면 좋겠다~       중립    -0.128     0.0 부정(0.591)        긍정     0.621 {'부정': 0.024, '중립': 0.355, '긍정': 0.621}


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import re

# ------------------------------------------------------------
# 1️⃣ 부정 렉시콘 확장
# ------------------------------------------------------------
negative_lexicon = [
    # 삭제 관련
    "작성자에 의해 삭제", "삭제된 댓글", "클린봇", "운영규정 미준수", "부적절한 표현", "검열", "비속어",
    # 불만/위약금/기업 비판
    "위약금", "손해배상", "환불", "보상", "배신", "기만", "피해", "불매", "해지", "사기",
    # 기업/이슈 관련 비판
    "통신사", "3사", "악당", "도둑", "쓰레기", "한심", "어이없", "뻔뻔", "불공정", "망했다",
    # 욕설/비하
    "개같", "ㅈㄴ", "ㅂㅅ", "씨발", "병신", "ㅗ", "죽어라", "미친", "좆", "꺼져",
    # 조롱/풍자
    "ㅋㅋ", "ㅎㅎ", "참", "또", "역시", "하하", "재밌네", "대단하네"
]

# ------------------------------------------------------------
# 2️⃣ 부정 감정 판정 함수
# ------------------------------------------------------------
def sentiment_from_lexicon(text, lexicon):
    text = str(text).lower()
    # 부정어 포함 여부 감지
    if any(word in text for word in lexicon):
        return -1  # 부정
    else:
        return 0   # 중립

# ------------------------------------------------------------
# 3️⃣ ALS용 평점 변환
# ------------------------------------------------------------
def map_sentiment_to_rating(sentiment):
    # -1(부정)=1점, 0(중립)=3점
    return 1 if sentiment == -1 else 3

# ------------------------------------------------------------
# 4️⃣ 댓글 데이터 예시 (엑셀 불러오기 가능)
# ------------------------------------------------------------
# 실제 데이터 불러오기
# df = pd.read_excel("C:/Users/speec/OneDrive/Desktop/PoC_v2/YouTube_결과/GS리테일_정보유출_comments_20251103_154830.xlsx")

# 샘플 예시
df = pd.DataFrame({
    "user_id": [1, 1, 2, 3],
    "item_id": [10, 11, 10, 12],
    "comment": [
        "작성자에 의해 삭제된 댓글입니다.",
        "위약금이 너무 심해요",
        "클린봇이 부적절한 표현을 감지한 댓글입니다.",
        "서비스가 좋아요 감사합니다"
    ]
})

# ------------------------------------------------------------
# 5️⃣ 감정 점수 및 평점 부여
# ------------------------------------------------------------
df["sentiment"] = df["comment"].apply(lambda x: sentiment_from_lexicon(x, negative_lexicon))
df["rating"] = df["sentiment"].apply(map_sentiment_to_rating)

# ------------------------------------------------------------
# 6️⃣ 결과 확인
# ------------------------------------------------------------
print(df[["user_id", "item_id", "comment", "sentiment", "rating"]])
